In [1]:
import cudf as cf

eurgbp = cf.read_csv('/home/edoardocame/Desktop/python_dir/data/eurgbp-m1-bid-2015-01-01-2025-01-30T14:40.csv', index_col=0)
eurgbp_after23 = eurgbp[eurgbp.index > '2023-01-01']

In [2]:
from cuml.linear_model import LinearRegression
import cupy as cp

# Create lagged price and compute the change (delta)
df_ou = eurgbp_after23[['close']].copy()
df_ou['lag'] = df_ou['close'].shift(1)
df_ou['delta'] = df_ou['close'] - df_ou['lag']
df_ou = df_ou.dropna()

# Convert cudf Series to cupy arrays for regression
X = df_ou['lag'].to_cupy().reshape(-1, 1)
y = df_ou['delta'].to_cupy()

# Fit the regression: delta = alpha + beta * lag
model = LinearRegression()
model.fit(X, y)
beta = model.coef_[0]

# Calculate the half-life of mean reversion
half_life = -cp.log(2) / beta
print("Half-life of mean reversion:", half_life.get())

Half-life of mean reversion: 50125.27560977508


/home/edoardocame/miniconda3/envs/rapids-23.12/lib/python3.10/site-packages/cuml/internals/api_decorators.py:382: UserWarning: Starting from version 23.08, the new 'copy_X' parameter defaults to 'True', ensuring a copy of X is created after passing it to fit(), preventing any changes to the input, but with increased memory usage. This represents a change in behavior from previous versions. With `copy_X=False` a copy might still be created if necessary. Explicitly set 'copy_X' to either True or False to suppress this warning.
  return init_func(self, *args, **filtered_kwargs)
/home/edoardocame/miniconda3/envs/rapids-23.12/lib/python3.10/site-packages/cuml/internals/api_decorators.py:190: UserWarning: Changing solver from 'eig' to 'svd' as eig solver does not support training data with 1 column currently.
  return func(*args, **kwargs)


In [3]:
import cudf
import cupy as cp


# Calculate the Hurst exponent using cupy
def compute_Hc_cupy(data, kind='price'):
    N = len(data)
    T = cp.arange(1, N + 1)
    Y = cp.cumsum(data - cp.mean(data))
    R = cp.max(Y) - cp.min(Y)
    S = cp.std(data)
    H = cp.log(R / S) / cp.log(T[-1])
    return H.get()

H_cupy = compute_Hc_cupy(cp.array(eurgbp_after23['close']))
print(f"Hurst exponent (cupy): {H_cupy}")

Hurst exponent (cupy): 0.9284973077652352
